# From comment_metadata.ipynb:

## Header Index Testing

In [13]:
with open('header_results.json') as f:
    header_results_json = json.load(f)

In [14]:
#tranlate ES keys to true rule section names, MAINTAINING ORDER
for key, value in header_results_json.items():
    value_list = []
    for val_idx in value:
        for true_key in truth_keys:
            if true_key in val_idx and true_key not in value_list: 
                value_list.append(true_key)
        header_results_json[key]=value_list     

In [ ]:
# match_dict shows all true positives, without penalty to false categorization
match_dict= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), header_results_json.items()):
    match_count = 0
    for val_idx in value1:
        if val_idx in value2:
            match_count += 1
        match_dict[key1]=match_count/len(value1)

In [ ]:
# unpenalized accuracy of true positives
meanvals = np.mean(list(match_dict.values()))
meanvals

In [ ]:
# match_dict1 shows all true positives, with penalty to false categorization
match_dict1= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), header_results_json.items()):
    match_count = 0
    for val_idx in value2:
        if val_idx in value1:
            match_count += 1
        match_dict1[key1]=match_count/len(value2)

In [ ]:
# penalized accuracy of true positives
meanvals1 = np.mean(list(match_dict1.values()))
meanvals1

In [ ]:
np.mean([meanvals,meanvals1])

### Sklearn NDCG Headers

In [ ]:
#create truth dict with values equal to sparse array length=29, binary
truth_dict_all = truth_dict.copy()
for key, value in truth_dict.items():
    new_value_list = np.zeros(len(truth_keys))
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                new_value_list[i] = 1
    truth_dict_all[key] = new_value_list

In [ ]:
#create header results dict with values equal to sparse array length=29, binary
binary_header_results_all = header_results_json.copy()
for key, value in header_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1
    binary_header_results_all[key] = new_value_list

In [ ]:
#create header results dict with values equal to sparse array length=29, (discounted)
header_results_all = header_results_json.copy()
for key, value in header_results_json.items():
    new_value_list = np.zeros(len(truth_keys))
    counter = 1
    for val in value:
        for i in range(len(truth_keys)):
            if val == truth_keys[i]:
                counter += 1
                new_value_list[i] = 1/math.log(counter, 2)
    header_results_all[key] = new_value_list

In [ ]:
headers_ndcg = {}
headers_ap = {}
headers_f1 = {}
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), header_results_all.items()):
    headers_ndcg[true_key] = ndcg_score([true_value], [es_value])
    headers_ap[true_key] = average_precision_score(true_value, es_value, pos_label=0)
for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), binary_header_results_all.items()):
    headers_f1[true_key] = f1_score(true_value, es_value, zero_division=0, pos_label=0)

In [ ]:
headers_eval = pd.DataFrame(data=[headers_ndcg, headers_ap, headers_f1]).T
headers_eval = headers_eval.rename({0:"NDCG", 1:"AP", 2:"F1"}, axis=1)

In [ ]:
plt.figure(figsize = (14,10))
sns.kdeplot(headers_eval.NDCG, alpha=.1, shade=True, label='NDCG')
sns.kdeplot(headers_eval.AP, alpha=.1, shade=True,label='AP')
sns.kdeplot(headers_eval.F1,  alpha=.1, shade=True,label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Headers Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

### NDCG

In [ ]:
def Cummulative_gain(truth, pred):
    counter = 0
    for i in truth:
        if i in pred:
            counter +=1
    return counter
def Discounted_Cummulative_gain(truth, pred):
    DCG = 0
    for i,j in enumerate(pred):
        if j in truth:
            DCG += (1/(math.log(i+2,2)))
    return DCG
def Ideal_discounted_cummulative_gain(truth, pred):
    counter = 0
    IDCG = 0
    for i in truth:
        if i in pred:
            counter +=1
    for i in range(counter):
        IDCG += (1/(math.log(i+2,2)))
    return IDCG
def nDiscounted_Cummulative_gain(truth, pred):
    x = Ideal_discounted_cummulative_gain(truth, pred)
    y = Discounted_Cummulative_gain(truth, pred)
    return y/(x+0.0001)

## Average Precision

In [ ]:
def average_precision(truth, pred):
    runn = 0
    cum_gain = 0
    for i, j in enumerate(truth):
        if j in pred:
            cum_gain +=1
            runn = runn + cum_gain/(i+1)
    average_pred = runn / (cum_gain + 0.0001)
    return average_pred

## F1 Score

In [ ]:
def F_1_score(truth, pred):
    AP = average_precision(truth, pred)
    F_1 = AP/(AP + 1)
    return 2*F_1

## Headers Scoring

In [ ]:
CG = {}
DCG = {}
iDCG = {}
nDCG = {}
AP = {}
F_1 = {}
MAP = 0
count = 0
for (i,(k,j)) in zip(truth_dict.values(),header_results_json.items()):
    CG[k] = Cummulative_gain(i,j)
    DCG[k] = Discounted_Cummulative_gain(i,j)
    iDCG[k] = Ideal_discounted_cummulative_gain(i,j)
    nDCG[k] = nDiscounted_Cummulative_gain(i,j)
    AP[k] = average_precision(i,j)
    F_1[k] = F_1_score(i,j)
    count +=1
    MAP +=AP[k]
Model_eval = pd.DataFrame(data=[CG, DCG, iDCG, nDCG, AP, F_1])
MAP/count

In [ ]:
Model_eval = Model_eval.transpose().rename({0:'CG',1:'DCG',2:'iDCG',3:'nDCG',4:'AP',5:'F1'}, axis=1)
Model_eval

In [ ]:
Mean_scores = Model_eval.mean()
Mean_scores

In [ ]:
plt.figure(figsize = (14,10))

sns.kdeplot(Model_eval.nDCG, alpha=.1, shade=True, label='nDCG')
sns.kdeplot(Model_eval.AP, alpha=.1, shade=True,label='AP')
sns.kdeplot(Model_eval.F1,  alpha=.1, shade=True,label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Headers Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

## Hybrid Scoring

In [ ]:
CG = {}
DCG = {}
iDCG = {}
nDCG = {}
AP = {}
F_1 = {}
MAP = 0
count = 0
for (i,(k,j)) in zip(truth_dict.values(),hybrid_results_json.items()):
    CG[k] = Cummulative_gain(i,j)
    DCG[k] = Discounted_Cummulative_gain(i,j)
    iDCG[k] = Ideal_discounted_cummulative_gain(i,j)
    nDCG[k] = nDiscounted_Cummulative_gain(i,j)
    AP[k] = average_precision(i,j)
    F_1[k] = F_1_score(i,j)
    count += 1
    MAP += AP[k]
Model_eval2 = pd.DataFrame(data=[CG, DCG, iDCG, nDCG, AP, F_1])
MAP/count

In [ ]:
Model_eval2 = Model_eval2.transpose().rename({0:'CG',1:'DCG',2:'iDCG',3:'nDCG',4:'AP',5:'F1'}, axis=1)
Model_eval2

In [ ]:
Mean_scores2 = Model_eval2.mean()
Mean_scores2

In [ ]:
mean_df = pd.DataFrame(data=[Mean_scores, Mean_scores2])
mean_df=mean_df.rename({0:'headers',1:'hybrid'},axis=0)
mean_df

In [ ]:
plt.figure(figsize = (14,10))

sns.kdeplot(Model_eval2.nDCG, alpha=.1, shade=True, label='nDCG')
sns.kdeplot(Model_eval2.AP, alpha=.1, shade=True, label='AP')
sns.kdeplot(Model_eval2.F1,  alpha=.1, shade=True, label='F1')
plt.legend()
plt.title('Ranking Metric Performance, Hybrid Index', fontsize=20)
plt.xlabel('Score Distribution', fontsize=16)
plt.ylabel('Score Density', fontsize=16)
plt.xlim(0,1);

# From es_upload.ipynb

In [ ]:
#Updated Character-Count Split

def splitRule_line_chars(rule_url):
    #get rule
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings

    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #Split on new paragraphs
    paragraphs = splitlist.split('\r\n')
    
    #add new lines while under 2000 characters
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs)-1) and len(paragraphs[i]) < 2000:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])

    #add to dictionary
    rulechunks = {}
    keys = range(len(paragraphs))
    for i in keys:
        rulechunks[i] = {"text": paragraphs[i]}

    return rulechunks

In [ ]:
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    #chunklist = []
    
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks

In [ ]:
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value["text"]) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value["text"][i:i+25000] for i in range(0, len(value["text"]), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = {"text": chunklist[i]} #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks

In [ ]:
def ruletoES_hybrid(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
        for small_key, small_value in value.items():
            res = es.index(index=es_index, id=idnum, body={small_key:small_value})
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

#### CHARCTER Full dictionary Query

In [ ]:
# CHARACTER Full dictionary Query
char_results = run_large_query(comments2018, "char_25000")

In [ ]:
# Capture CHARACTER results in CSV
w = csv.writer(open('char_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
# Capture CHARACTER results in JSON
json = json.dumps(results)
f = open('char_results.json','w')
f.write(json)
f.close()

#### HEADERS Full dictionary Query

In [ ]:
header_results = run_large_query(comments2018, "headers_new")

In [ ]:
w = csv.writer(open('header_results2.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
json = json.dumps(results)
f = open('header_results2.json','w')
f.write(json)
f.close()

#### HYBRID Full dictionary Query

In [ ]:
hybrid_results = run_large_query(comments2018, "hybrid_6000")

In [ ]:
# Capture HYBRID results in CSV
w = csv.writer(open('hybrid_results.csv', 'w'))
for key, val in hybrid_results.items():
    w.writerow([key,val])

In [ ]:
# Capture HYBRID results in JSON
json = json.dumps(hybrid_results)
f = open('hybrid_results.json','w')
f.write(json)
f.close()